### Tree collapsing
Tristan Miller

Just a short demo to show how we can adjust the tolerance for tree collapsing

In [1]:
import pandas as pd
import numpy as np
from cluster_tree import *

In [2]:
from ete3 import Tree, TreeStyle
ts = TreeStyle()
ts.show_leaf_name = True
ts.scale =  12

In [3]:
ward_tree = pd.read_pickle("uncollapsed_tree.pkl")
c_labels = pd.read_pickle("c_labels.pkl")
doc_term = pd.read_pickle("Tfidf_Matrix.pkl")
term_list = pd.read_pickle("Feature_List.pkl")

In [6]:
tolerance=-0.3
cmeans_terms,ndocs = get_means(c_labels,doc_term[:25000,:])
collapsed_tree = collapse_label_tree(ward_tree,cmeans_terms,ndocs,tolerance)
name_tree = get_name_tree(collapsed_tree,cmeans_terms,ndocs,term_list)
t=Tree(str(name_tree))
t.show(tree_style=ts)

In [5]:
print(ndocs)

[  580.   458.   274.   271.   180.   251.  6318.   893.   917.   159.
   297.   250.   170.   182.   163.   497.   368.   543.   239.   459.
   201.    79.   144.    77.   155.   122.   297.    75.   360.    74.
   240.    94.   149.   447.   114.   328.   244.   165.   178.   284.
   110.   161.   142.    72.   132.    73.   115.   122.   148.    76.
    48.   210.   140.   113.   183.   254.   169.    68.   104.   163.
    71.   190.   164.   306.    56.    55.    83.   189.   173.    74.
   273.   161.    80.    65.    89.    53.   103.   132.   252.   296.
    69.   309.    58.   113.    79.   120.    51.   418.   102.    54.
    74.   145.   100.   104.    81.    45.    99.    72.    84.    59.]


By far the largest cluster is the one that links to ISBN.  LOL